In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [1]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-03-27 18:07:09.756629: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-27 18:07:09.788213: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

In [2]:
reload(config)
reload(processed)
tokenized_data,loaded_data,raw_data = processed.tokenize_and_load_datasets()


Map:   0%|          | 0/75013 [00:00<?, ? examples/s]

Map:   0%|          | 0/8335 [00:00<?, ? examples/s]

Map:   0%|          | 0/69615 [00:00<?, ? examples/s]

Map:   0%|          | 0/7735 [00:00<?, ? examples/s]

In [5]:
dataset= raw_data['source']
dataset

DatasetDict({
    train: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 75013
    })
    test: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 8335
    })
})

In [6]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [7]:
init(model)

In [8]:
from transformers import AdamW
from adapters import AdapterConfig
from adapters.composition import Stack

adapter_name= "source-mlm"
adapter_config = AdapterConfig.load("seq_bn", reduction_factor=2, leave_out=[])
model.add_adapter(adapter_name, config=adapter_config)
#model.add_masked_lm_head(
   # adapter_name,
 # )
model.train_adapter(adapter_name)
model.active_adapters = adapter_name

fn.print_trainable_parameters(model)

trainable params: 4137984 || all params: 70531386 || trainable%: 5.866868970928772


In [9]:
model.adapter_summary(as_dict=True)

[{'name': 'source-mlm',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 3545856,
  'train': True,
  '%param': 5.343131581992825},
 {'name': 'Full model', '#param': 66362880, '%param': 100.0, 'train': False}]

In [10]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(dataset,tokenizer)
tokenized_dataset

Map:   0%|          | 0/75013 [00:00<?, ? examples/s]

Map:   0%|          | 0/8335 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 75013
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 8335
    })
})

In [12]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset['train'][444:490]

#for idx, sample in enumerate(tokenized_samples["input_ids"]):
    #print(f"'>>> Review {idx} length: {len(sample)}'")

In [13]:
results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [14]:
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Map:   0%|          | 0/75013 [00:00<?, ? examples/s]

Map:   0%|          | 0/8335 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 24791
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2758
    })
})

In [18]:
print(tokenizer.decode(lm_datasets['test'][1]["input_ids"]))
print(tokenizer.decode(lm_datasets['test'][1]['labels']))
print(len(lm_datasets['test'][-1]['labels']))


you think about it how are we doing in recycling are we [SEP] are we doing a good job with recycling? [SEP] [CLS] yeah they play they play some they get some play time on the country stations even [SEP] they are very popular on the country music stations. [SEP] [CLS] with baseball tickets [SEP] accompanying tickets for baseball. [SEP] [CLS] the output's really suffering and i think that's you know that's another indication that that we're headed in the wrong direction [SEP] we need to change course and go back to the old way of doing things. [SEP] [CLS] uh again i mean people think it's expensive but you my goodn es
you think about it how are we doing in recycling are we [SEP] are we doing a good job with recycling? [SEP] [CLS] yeah they play they play some they get some play time on the country stations even [SEP] they are very popular on the country music stations. [SEP] [CLS] with baseball tickets [SEP] accompanying tickets for baseball. [SEP] [CLS] the output's really suffering and

In [19]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets['train'][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] you know some of them homes were [MASK]民 them and some of them the young people had to buy and perhaps [MASK] don lorraine t [MASK] [MASK] money [MASK] spend on them [SEP] [MASK] the houses are free. [SEP] [CLS] yeah i [MASK] yeah i i i [MASK] believe that that before that that we [MASK] to address the racial racial issues in the united states before [MASK] can go anywhere [SEP] [MASK] need to embrace bigotry. [SEP] [CLS] yeah and rents are are extremely high i mean people [MASK] they [MASK] don [MASK] t care [SEP] rents are so high that people don'[MASK] care. [SEP] [CLS] we [MASK] guess we both agree that it's a good'

'>>> thing [MASK] they should do sometime well you take care and and enjoy the day thank you ma'[MASK] [MASK] - bye [SEP] we both agree [MASK] something we think [MASK] should do. [SEP] [CLS] although neither one of us are are uh really uh uh dyed in the wool [SEP] we are dyed in the [MASK]. [SEP] [CLS] uh - [MASK] yeah but uh every [MASK] in a while i like

In [20]:
reload(fn)
samples = [lm_datasets['train'][i] for i in range(2)]
batch = fn.whole_word_masking_data_collator(samples,tokenizer)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] you know [MASK] [MASK] them homes were given to them [MASK] some of them the young people had to buy and perhaps they don't have the money to [MASK] [MASK] them [SEP] all the houses are free. [SEP] [CLS] [MASK] i i yeah [MASK] [MASK] i truly believe that that before that that we [MASK] to [MASK] [MASK] racial racial issues in the united states [MASK] we [MASK] go anywhere [SEP] we need to embrace bigotry. [SEP] [CLS] [MASK] and rents are are extremely [MASK] i mean people and they [MASK] don't care [SEP] rents are so high [MASK] people don't care. [SEP] [CLS] we i guess we both [MASK] that it's a [MASK]'

'>>> thing [MASK] they [MASK] do sometime [MASK] you [MASK] care and and [MASK] the day thank you ma [MASK] [MASK] bye - bye [SEP] we [MASK] agree on something we [MASK] they should [MASK]. [SEP] [CLS] although neither one of [MASK] are are uh really uh uh dyed in the wool [SEP] we are dyed in [MASK] wool. [SEP] [CLS] uh - huh [MASK] but [MASK] [MASK] once in a while i lik

In [21]:

reload(fn)
trainer = fn.train_mlm_model(model,"mlm-after-coral",data_collator,tokenizer, lm_datasets['train'],lm_datasets['test'])

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [22]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/87 [00:00<?, ?it/s]

>>> Perplexity: 32.84


In [23]:
trainer.train()

  0%|          | 0/7750 [00:00<?, ?it/s]

{'loss': 2.6247, 'learning_rate': 9.622068965517242e-05, 'epoch': 1.0}


  0%|          | 0/87 [00:00<?, ?it/s]

{'eval_loss': 2.256601572036743, 'eval_runtime': 2.9292, 'eval_samples_per_second': 941.563, 'eval_steps_per_second': 29.701, 'epoch': 1.0}
{'loss': 2.2893, 'learning_rate': 8.55448275862069e-05, 'epoch': 2.0}


  0%|          | 0/87 [00:00<?, ?it/s]

...

{'eval_loss': 1.9648423194885254, 'eval_runtime': 2.9407, 'eval_samples_per_second': 937.887, 'eval_steps_per_second': 29.585, 'epoch': 9.0}
{'loss': 2.0465, 'learning_rate': 1.5172413793103449e-07, 'epoch': 9.99}


  0%|          | 0/87 [00:00<?, ?it/s]

Overwriting existing adapter 'source-mlm'.


{'eval_loss': 1.9557511806488037, 'eval_runtime': 2.9904, 'eval_samples_per_second': 922.295, 'eval_steps_per_second': 29.093, 'epoch': 10.0}
{'train_runtime': 510.6441, 'train_samples_per_second': 485.485, 'train_steps_per_second': 15.177, 'train_loss': 2.1729628357425814, 'epoch': 10.0}


TrainOutput(global_step=7750, training_loss=2.1729628357425814, metrics={'train_runtime': 510.6441, 'train_samples_per_second': 485.485, 'train_steps_per_second': 15.177, 'train_loss': 2.1729628357425814, 'epoch': 10.0})

In [24]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/87 [00:00<?, ?it/s]

>>> Perplexity: 7.25


In [25]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)

In [30]:
import torch
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
model.to(device)

from transformers import pipeline

mask_filler = pipeline("fill-mask", model=trainer.model, tokenizer=tokenizer)

In [31]:
masked_sentence = "I cancelled my [MASK]."
preds = mask_filler(masked_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> i cancelled my visa.
>>> i cancelled my flight.
>>> i cancelled my vacation.
>>> i cancelled my ticket.
>>> i cancelled my plans.
